# Simple network

This network is composed of 3 machines: Starting_Client, Backup_Server, Active_Directory (final target).

The process that the attack needs to take to arrive at the final target is the following:

```
Starting_Client -> Discover Backup_Server (Exploration) -> Discover Hard Coded creds for Backup_Server -> Infect Backup_Server (Lateral movement) -> Get admin access to backup server (Vertical movement) ->  Discover Active_Directory -> Infect Active Directory (Lateral movement). 
```

In [ ]:
import sys, logging
import cyberbattle.simulation.model as model
import cyberbattle.simulation.commandcontrol as commandcontrol
import cyberbattle.networks.simple_network as sn
import plotly.offline as plo

plo.init_notebook_mode(connected=True) # type: ignore
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(levelname)s: %(message)s")
%matplotlib inline

In [ ]:
network = model.create_network(sn.nodes)
env = model.Environment(network=network, vulnerability_library=dict([]), identifiers=sn.ENV_IDENTIFIERS)
env

In [ ]:
c2 = commandcontrol.CommandControl(env)
dbg = commandcontrol.EnvironmentDebugging(c2)


def plot():
    dbg.plot_discovered_network()
    c2.print_all_attacks()


# plot()

## Discover Backup_Server

In [ ]:
outcome = c2.run_attack("Starting_Client", "ReadSourceCode_LeakedNode")
plot()

## Discover Hard Coded creds for Backup_Server

In [ ]:
outcome = c2.run_attack("Starting_Client", "ReadSourceCode_LeakedCreds")
plot()
c2.credentials_gathered_so_far

## Infect Backup_Server (Lateral movement)

In [ ]:
backup_server = c2.connect_and_infect(
    source_node_id="Starting_Client",
    target_node_id="Backup_Server",
    credentials="BckUserCreds",
    port_name="SSH"
)
plot()

## Get admin access to backup server (Vertical movement) 

In [ ]:
outcome = c2.run_attack("Backup_Server", "WritableBackupScript")
plot()

## Discover Active_Directory 

In [ ]:
outcome = c2.run_attack("Backup_Server", "ReadScriptLogs")
plot()

## Infect Active Directory (Lateral movement). 


In [ ]:
c2.run_remote_attack("Backup_Server", "Active_Directory", "Remote_RDP_Exploit")
plot()